# 为情感分析和词嵌入创建 Yelp 评论数据

## 导入和设置

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd
from pandas.io.json import json_normalize

## 关于数据

数据由多个文件组成，其中包含 Yelp 为鼓励数据科学创新而提供的有关业务、用户、评论和其他方面的信息。

数据由多个文件组成，其中包含 Yelp 为鼓励数据科学创新而提供的有关业务、用户、评论和其他方面的信息。

我们将使用 2010-2019 年期间产生的大约 600 万条评论来提取文本特征。此外，我们将使用与用户评论一起提交的其他信息。

## 获取数据

接受许可后，您可以从 [此处](https://www.yelp.com/dataset) 下载 json 格式的数据。 2020 版本有 4.7GB（压缩）和大约 10.5GB（未压缩）的文本数据。

下载后, 将五个 .json 文件中的以下两个解压到 ./yelp/json 中：
- the `yelp_academic_dataset_user.json`
- the `yelp_academic_dataset_reviews.json`

通过去除 `yelp_academic_dataset_` 前缀来重命名这两个文件，以便您具有以下目录结构：
```
data
|-create_yelp_review_data.ipynb
|-yelp
    |-json
        |-user.json
        |-review.json
```


In [3]:
yelp_dir = Path('yelp')

if not yelp_dir.exists():
    yelp_dir.mkdir(exist_ok=True)

## 解析 json 并存储为 parquet 文件

将 json 转换为更快的parquet 格式：


In [4]:
for fname in ['review', 'user']:
    print(fname)
    
    json_file = yelp_dir / 'json' / f'{fname}.json'
    parquet_file = yelp_dir / f'{fname}.parquet'
    if parquet_file.exists():
        print('\talready exists')
        continue

    data = json_file.read_text(encoding='utf-8')
    json_data = '[' + ','.join([l.strip()
                                for l in data.split('\n') if l.strip()]) + ']\n'
    data = json.loads(json_data)
    df = json_normalize(data)
    if fname == 'review':
        df.date = pd.to_datetime(df.date)
        latest = df.date.max()
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df = df.drop(['date', 'business_id', 'review_id'], axis=1)
    if fname == 'user':
        df.yelping_since = pd.to_datetime(df.yelping_since)
        df = (df.assign(member_yrs=lambda x: (latest - x.yelping_since)
                        .dt.days.div(365).astype(int))
              .drop(['elite', 'friends', 'name', 'yelping_since'], axis=1))
    df.dropna(how='all', axis=1).to_parquet(parquet_file)

review
user


现在您可以删除 json 文件。

In [8]:
def merge_files(remove=False):
    combined_file = yelp_dir / 'user_reviews.parquet'
    if not combined_file.exists():
        user = pd.read_parquet(yelp_dir / 'user.parquet')
        print(user.info(null_counts=True))

        review = pd.read_parquet(yelp_dir / 'review.parquet')
        print(review.info(null_counts=True))

        combined = (review.merge(user, on='user_id',
                                 how='left', suffixes=['', '_user'])
                    .drop('user_id', axis=1))
        combined = combined[combined.stars > 0]
        print(combined.info(null_counts=True))
        combined.to_parquet(yelp_dir / 'user_reviews.parquet')
    else:
        print('already merged')
    if remove:
        for fname in ['user', 'review']:
            f = yelp_dir / (fname + '.parquet')
            if f.exists():
                f.unlink()

In [9]:
merge_files(remove=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968703 entries, 0 to 1968702
Data columns (total 19 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1968703 non-null  object 
 1   review_count        1968703 non-null  int64  
 2   useful              1968703 non-null  int64  
 3   funny               1968703 non-null  int64  
 4   cool                1968703 non-null  int64  
 5   fans                1968703 non-null  int64  
 6   average_stars       1968703 non-null  float64
 7   compliment_hot      1968703 non-null  int64  
 8   compliment_more     1968703 non-null  int64  
 9   compliment_profile  1968703 non-null  int64  
 10  compliment_cute     1968703 non-null  int64  
 11  compliment_list     1968703 non-null  int64  
 12  compliment_note     1968703 non-null  int64  
 13  compliment_plain    1968703 non-null  int64  
 14  compliment_cool     1968703 non-null  int64  
 15  compliment_funn

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'